In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics supervision roboflow

from ultralytics import YOLO
from roboflow import Roboflow


In [ ]:
from IPython import display
display.clear_output()
from IPython.display import display, Image


import ultralytics
ultralytics.checks()

# Original YOLO11s execution

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Access key can be provided upon request.")
project = rf.workspace("tamilarasi-dpda3").project("brinjal-detection-5niwn")
version = project.version(7)
dataset = version.download("yolov11")


In [ ]:
!sed -i '$d' {dataset.location}/data.yaml   # Delete the last line
!sed -i '$d' {dataset.location}/data.yaml   # Delete the second-to-last line
!sed -i '$d' {dataset.location}/data.yaml   # Delete the third-to-last line

!echo 'test: ../test/images' >> {dataset.location}/data.yaml
!echo 'train: ../train/images' >> {dataset.location}/data.yaml
!echo 'val: ../valid/images' >> {dataset.location}/data.yaml


In [ ]:
model11=YOLO("yolo11s.pt")


In [ ]:
from sklearn.model_selection import KFold
from ultralytics import YOLO
import os
import shutil
from glob import glob
from pathlib import Path # Import Path

# Parameters
k = 5
dataset_dir = Path('/content/Brinjal-detection-7' ) # DATASET PATH
images = sorted(glob(os.path.join(dataset_dir, 'images', '*.jpg')))
labels = sorted(glob(os.path.join(dataset_dir, 'labels', '*.txt')))
assert len(images) == len(labels)
print(dataset_dir)


dataset_path = Path("/content/Brinjal-detection-7")  # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'
images = sorted(dataset_path.rglob("*images/*.jpg"))  # all data in 'images'
print(len(images))

# K-Fold
kf = KFold(n_splits=k, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(images)):
    print(f'Fold {fold + 1}')

    # Prepare directories
    fold_dir = f'fold_{fold + 1}'
    for split in ['train', 'val']:
        for subdir in ['images', 'labels']:
            os.makedirs(os.path.join(fold_dir, split, subdir), exist_ok=True)

    # Copy training data
    for idx in train_idx:
        shutil.copy(images[idx], os.path.join(fold_dir, 'train', 'images'))
        shutil.copy(labels[idx], os.path.join(fold_dir, 'train', 'labels'))

    # Copy validation data
    for idx in val_idx:
        shutil.copy(images[idx], os.path.join(fold_dir, 'val', 'images'))
        shutil.copy(labels[idx], os.path.join(fold_dir, 'val', 'labels'))

    # Create YOLO data config file
    data_yaml = f"""
path: {os.path.abspath(fold_dir)}
train: train/images
val: val/images
names: ['Brinjal']  # Adjust to your classes
    """

    with open(os.path.join(fold_dir, 'data.yaml'), 'w') as f:
        f.write(data_yaml)

    # Train YOLOv8
    model = YOLO('yolo11s.pt')  # or yolov8s.pt / custom checkpoint
    model.train(data=os.path.join(fold_dir, 'data.yaml'), epochs=50, imgsz=640)